In [13]:
import sqlite3
from autogc_validation.database.database.schema.create_table import create_table
from autogc_validation.database.database.schema.schemas import SCHEMAS
from autogc_validation.database.utils.connection import get_connection
import pandas as pd
import logging
import tempfile
import os


In [14]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [16]:
with tempfile.NamedTemporaryFile(suffix=".db", delete=False) as tmpfile:
    db_path = tmpfile.name

logger.info(f"Temporary database path: {db_path}")

def show_tables(conn):
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    return [row[0] for row in cursor.fetchall()]

# -----------------------------
# Create tables and inspect
# -----------------------------
with get_connection(db_path) as conn:
    # Create tables from SCHEMAS
    for table_name, sql in SCHEMAS.items():
        logger.info(f"Creating table: {table_name}")
        conn.execute(sql.sql)

    tables = show_tables(conn)
    print("Tables in DB:", tables)

# -----------------------------
# Optional: Clean up
# -----------------------------
os.remove(db_path)
logger.info("Temporary database deleted.")

INFO:__main__:Temporary database path: C:\Users\AENGST~1\AppData\Local\Temp\tmptch3rrd4.db
INFO:__main__:Creating table: schema_version
INFO:__main__:Creating table: voc_info
INFO:__main__:Temporary database deleted.


Tables in DB: ['SchemaVersion', 'voc_info']
